In [ ]:
# Train 1 : 30k iterations ,ADAM default lr, no weights on losses, iloss stuck at 50 fail
# Train 2 : 70k iterations ,ADAM default lr, weight 100 on iloss and bloss, iloss stuck at 16 fail
# Train 3 : 1k iterations , L-BFGS default, weight 100 on iloss and bloss, iloss stuck at 150 fail
# Train 4 : 30k , 2k , ADAM default lr, L-BFGS default, weight 100 on iloss and bloss, iloss stuck at ~15
# Train 5 : 100k iteration , ADAM 0.005 lr,  weight 100 iloss bloss,

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import math
import matplotlib.pyplot as plt
import numpy as np


$$
\frac{\partial^2 NN(x,t)}{\partial t^2} = v^2 \frac{\partial^2 NN(x,t)}{\partial x^2}
$$
$$
\text{I.C} = NN(x,0) = sin(n\pi x) , NN_t(x,0) = 0
$$
$$
\text{B.C} = NN(0,t) =  NN(1,t) = 0
$$
$$
\text{Analytical Solution = }sin(\pi n x)cos(\pi v n t)
$$

In [ ]:
n = 2 #  harmonic
v = 1 # propagation speed
nf = 10000 # physics samples
nu = 100 # physics data


In [ ]:
x_values = torch.rand(nf,1,dtype=torch.float32,device="cuda",requires_grad=True) # # Using domain (0,1) to make it easier
t_values = torch.rand(nf,1,dtype=torch.float32,device="cuda",requires_grad=True) # Domain (0,1) should teach the PINN a period of second harmonic, 
                                                                                 #T = 2/n
x_t = torch.cat((x_values,t_values),1)

In [ ]:
print(x_values.max(),t_values.max())
print(x_values.min(),t_values.min())

In [ ]:
# I.C values / shape = nu/2 , 1
x_ic = torch.rand((nu//2),1,dtype=torch.float32,device="cuda",requires_grad=True)
t_ic = torch.zeros_like(x_ic,dtype=torch.float32,device="cuda",requires_grad=True)

x_t_ic = torch.cat((x_ic,t_ic),1) # (nu/2 , 2)

In [ ]:
# B.C values / shape = nu/4 ,1 / x 2
t_bc = torch.rand((nu//4),1,dtype=torch.float32,device="cuda")
x_bc_one = torch.ones_like(t_bc,dtype=torch.float32,device="cuda")
x_bc_zero = torch.zeros_like(t_bc,dtype=torch.float32,device="cuda")

t_bc_one = torch.cat((x_bc_one,t_bc),1) # NN(1,t)
t_bc_zero = torch.cat((x_bc_zero,t_bc),1) # NN(0,t)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2,40)
        self.fc2 = nn.Linear(40,40)
        self.fc3 = nn.Linear(40,40)
        self.fc4 = nn.Linear(40,40)   # Architecture given by Raissi et al.
        self.fc5 = nn.Linear(40,40)
        self.fc6 = nn.Linear(40,1)
    def forward(self,x):
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.fc3(x)
        x = F.tanh(x)
        x = self.fc4(x)
        x = F.tanh(x)
        x = self.fc5(x)
        x = F.tanh(x)
        x = self.fc6(x)
        return x

In [ ]:
pinner = Model().to("cuda")

In [ ]:
optimizer = torch.optim.LBFGS(pinner.parameters()) # L-BFGS is the optimizer of choice in Raissi's paper
optimizer2 = torch.optim.Adam(pinner.parameters(),lr= 0.005)
w_iloss = 100
w_bloss = 100
w_ploss = 1

In [ ]:
ilosslist = []
blosslist = []
plosslist= []
total_losslist = []

In [ ]:
def train(model,xs,ts,optimizer,iteration):
    for i in tqdm(range(iteration)):            
        model.zero_grad()
        xt_f = torch.cat((xs, ts), dim=1)
        outs = model(xt_f) # (nf,1)

        # Boundary Loss
        bloss = (model(t_bc_one) ** 2).mean() + ((model(t_bc_zero) ** 2)).mean() # scalar

        # I.C Loss
        icx0 = model(x_t_ic) # (nf/2 , 1)
        dNN_dt = torch.autograd.grad(icx0,t_ic,grad_outputs=torch.ones_like(icx0),retain_graph=True)[0]
        
        iloss = ((icx0 - (torch.sin(n*torch.pi*x_ic)))**2).mean() + ((dNN_dt)**2).mean() #scalar

        # Physics Loss
        dnndt = torch.autograd.grad(outs,ts,grad_outputs=torch.ones_like(outs),create_graph=True)[0]
        dnndx = torch.autograd.grad(outs,xs,grad_outputs=torch.ones_like(outs),create_graph=True)[0]

        d2nndt2 = torch.autograd.grad(dnndt,ts,grad_outputs=torch.ones_like(outs),create_graph=True)[0]
        d2nndx2 = torch.autograd.grad(dnndx,xs,grad_outputs=torch.ones_like(outs),create_graph=True)[0]

        ploss = ((((d2nndx2*v*v) - (d2nndt2))) **2).mean()

        # Total loss
        losstotal = bloss*w_bloss + iloss*w_iloss + ploss*w_ploss # scalar
        losstotal.backward()
        optimizer.step()
        if i % 100 == 0:
            ilosslist.append(iloss.item())
            blosslist.append(bloss.item())
            plosslist.append(ploss.item())
            total_losslist.append(losstotal.item())
            


In [ ]:
# L - BFGS train func
def train2(model,xs,ts,optimizer,iteration):
    for i in tqdm(range(iteration)):
        def closure():
            model.zero_grad()
            xt_f = torch.cat((xs, ts), dim=1)
            outs = model(xt_f) # (nf,1)
    
            # Boundary Loss
            bloss = (model(t_bc_one) ** 2).mean() + ((model(t_bc_zero) ** 2)).mean() # scalar
    
            # I.C Loss
            icx0 = model(x_t_ic) # (nf/2 , 1)
            dNN_dt = torch.autograd.grad(icx0,t_ic,grad_outputs=torch.ones_like(icx0),retain_graph=True)[0]
            
            iloss = ((icx0 - (torch.sin(n*torch.pi*x_ic)))**2).mean() + ((dNN_dt)**2).mean() #scalar
    
            # Physics Loss
            dnndt = torch.autograd.grad(outs,ts,grad_outputs=torch.ones_like(outs),create_graph=True)[0]
            dnndx = torch.autograd.grad(outs,xs,grad_outputs=torch.ones_like(outs),create_graph=True)[0]
    
            d2nndt2 = torch.autograd.grad(dnndt,ts,grad_outputs=torch.ones_like(outs),create_graph=True)[0]
            d2nndx2 = torch.autograd.grad(dnndx,xs,grad_outputs=torch.ones_like(outs),create_graph=True)[0]
    
            ploss = ((((d2nndx2*v*v) - (d2nndt2))) **2).mean()
    
            # Total loss
            losstotal = bloss*100 + iloss*100 + ploss # scalar
            losstotal.backward()
            if i % 100 == 0:
                ilosslist.append(iloss.item())
                blosslist.append(bloss.item())
                plosslist.append(ploss.item())
                total_losslist.append(losstotal.item())
            return losstotal
                
        optimizer.step(closure)



In [ ]:
train(pinner,x_values,t_values,optimizer2,50000)

In [ ]:
#train2(pinner,x_values,t_values,optimizer,3000)

In [ ]:
plt.plot(ilosslist)
plt.savefig("train_5/iloss5.png")

In [ ]:
plt.plot(blosslist)
plt.savefig("train_5/bloss5.png")

In [ ]:
plt.plot(plosslist)
plt.savefig("train_5/ploss5.png")

In [ ]:
plt.plot(total_losslist)
plt.savefig("train_5/total_loss5.png")

In [ ]:
#gemini
# 100 points for time (t)
t_test = torch.linspace(0, 1, 100).to("cuda")
# 201 points for space (x)
x_test = torch.linspace(0, 1, 201).to("cuda")

# Use meshgrid to get all (x, t) pairs
grid_x, grid_t = torch.meshgrid(x_test, t_test, indexing='ij')

# Flatten them into [N, 2] shape for the model
xt_test = torch.stack((grid_x.flatten(), grid_t.flatten()), dim=1)
pinner.eval() # Set model to evaluation mode
with torch.no_grad(): # No need to track gradients
    u_pred = pinner(xt_test)

# Reshape the flat [N, 1] output back into a 2D grid [201, 100]
u_pred_grid = u_pred.reshape(201, 100)
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Convert to numpy for plotting
u_data = u_pred_grid.cpu().numpy()

fig, ax = plt.subplots()
ax.set_ylim(-2.5, 2.5) # Set y-axis limits
line, = ax.plot(x_test.cpu().numpy(), u_data[:, 0]) # Plot first frame

def animate(frame):
    line.set_ydata(u_data[:, frame]) # Update the y-data for each frame
    return line,

ani = FuncAnimation(fig, animate, frames=100, blit=True)
ani.save("train_5/wave5.gif")
# (To save as a gif, you'd use ani.save('wave.gif'))